In [28]:
include("graph.jl")
include("base_operators.jl")

include("Convolution.jl")
include("Dense.jl")
include("Flatten.jl")
include("MaxPool.jl")
include("ReLu.jl")

include("misc.jl")

onecold (generic function with 1 method)

In [2]:
# 
import Pkg
Pkg.add("BenchmarkTools")
Pkg.add("MLDatasets")
Pkg.add("ProgressMeter")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [29]:
using MLDatasets;
using BenchmarkTools;
using Random;
using ProgressMeter;

In [30]:
train_log = []
settings = (;
    eta = Float32(1e-2),
    epochs = 3,
    batchsize = 100
)

(eta = 0.01f0, epochs = 3, batchsize = 100)

In [31]:
train_dataset = MNIST(:train);
train_samples = size(train_dataset.features)[3];
test_dataset = MNIST(:test);
test_samples = size(test_dataset.features)[3];

In [32]:
# obraz wejściowy i rzeczywista wartość
img = Variable(train_dataset[1].features, name="img")
actual_class = Variable(train_dataset[1].targets, name="actual_class")

var actual_class
 ┣━ value:    Int64
 ┗━ gradient: Nothing

In [6]:
function net_hard(x, w1, w2, w3, w4, b1, b2, b3, b4, y)
    o1 = Conv(x, w1, b1, ReLu)
    o2 = MaxPool(o1, Constant((2,2)))
    o3 = Conv(o2, w2, b2, ReLu)
    o4 = MaxPool(o3, Constant((2,2)))
    o5 = Flatten(o4)
    o6 = Dense(w3, b3, o5, ReLu)
    o7 = Dense(w4, b4, o6, Softmax)
    E = cross_entropy_loss(o7, y)

    return topological_sort(E), o7
end

w1 = Variable(glorot_uniform((3,3,1,6)), name="w1")
b1 = Variable(zeros(Float32, 6), name="b1")

w2 = Variable(glorot_uniform((3,3,6,16)), name="w2")
b2 = Variable(zeros(Float32, 16), name="b2")

w3 = Variable(glorot_uniform((84, 400)), name = "w3")
b3 = Variable(zeros(Float32, 84), name="b3")

w4 = Variable(glorot_uniform((10, 84)), name = "w4")
b4 = Variable(zeros(Float32, 10), name="b4")


graph, y_output = net_hard(img, w1, w2, w3, w4, b1, b2, b3, b4, actual_class)

var actual_class
 ┣━ value:    Int64
 ┗━ gradient: Nothing

In [35]:
# function net_easy(x, w1, w2, w3, y)
#     o1 = Conv(x, w1, ReLu)
#     o2 = MaxPool(o1, Constant((2,2)))
#     o3 = Flatten(o2)
#     o4 = Dense(w2, o3, ReLu)
#     o5 = Dense(w3, o4, Softmax)
#     E = cross_entropy_loss(o5, y)

#     return topological_sort(E), o5
# end

# w1 = Variable(glorot_uniform((3,3,1,6)), name="w1")
# w2 = Variable(glorot_uniform((84, 1014)), name="w2")   
# w3 = Variable(glorot_uniform((10, 84)), name="w3") 

# graph, y_output = net_easy(img, w1, w2, w3, actual_class)

(Any[var actual_class
 ┣━ value:    Int64
 ┗━ gradient: Nothing, op ?(typeof(-)), var w3
 ┣━ value:    10×84 Matrix{Float32}
 ┗━ gradient: Nothing, var w2
 ┣━ value:    84×1014 Matrix{Float32}
 ┗━ gradient: Nothing, var img
 ┣━ value:    28×28 Matrix{Float32}
 ┗━ gradient: Nothing, var w1
 ┣━ value:    3×3×1×6 Array{Float32, 4}
 ┗━ gradient: Nothing, op.?(typeof(Convolution)), op.?(typeof(ReLu)), const (2, 2), op.?(typeof(MaxPool)), op.?(typeof(Flatten)), op.?(typeof(mul!)), op.?(typeof(ReLu)), op.?(typeof(mul!)), op.?(typeof(Softmax)), op.?(typeof(log)), op.?(typeof(*)), op.?(typeof(sum))], op.?(typeof(Softmax)))

In [38]:
w1_grad = zeros(Float32, 3,3,1,6)
w2_grad = zeros(Float32, 3,3,6,16)
w3_grad = zeros(Float32, 84,400)
w4_grad = zeros(Float32, 10, 84)

b1_grad = zeros(Float32, 6)
b2_grad = zeros(Float32, 16)
b3_grad = zeros(Float32, 84)
b4_grad = zeros(Float32, 10)

# w1_grad = zeros(Float32, 3,3,1,6)
# w2_grad = zeros(Float32, 84, 1014)
# w3_grad = zeros(Float32, 10, 84)

for epoch in 1:settings.epochs
    train_losses = zeros(Float32, train_samples)
    train_acc = 0
    @time @showprogress dt=1 barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',) desc="Training..." for i in 1:train_samples
    #@time for i in 1:train_samples
            img.output = reshape(train_dataset[i].features, 28,28,1,1)
            actual_class.output = onehot(train_dataset[i].targets, 0:9)
            loss_value = forward!(graph)
            train_losses[i] = loss_value
			backward!(graph)
        
            onecold(y_output.output, 0:9) == train_dataset[i].targets ? train_acc+=1 : nothing
        
            w1_grad .+= w1.gradient
            w2_grad .+= w2.gradient
            w3_grad .+= w3.gradient
            # w4_grad .+= w4.gradient

            # b1_grad .+= b1.gradient
            # b2_grad .+= b2.gradient
            # b3_grad .+= b3.gradient
            # b4_grad .+= b4.gradient
            
            if i % settings.batchsize == 0
                w1.output = w1.output .- ((w1_grad/settings.batchsize)*settings.eta)
                w2.output = w2.output .- ((w2_grad/settings.batchsize)*settings.eta)
                w3.output = w3.output .- ((w3_grad/settings.batchsize)*settings.eta)
                w4.output = w4.output .- ((w4_grad/settings.batchsize)*settings.eta)

                b1.output = b1.output .- ((b1_grad/settings.batchsize)*settings.eta)
                b2.output = b2.output .- ((b2_grad/settings.batchsize)*settings.eta)
                b3.output = b3.output .- ((b3_grad/settings.batchsize)*settings.eta)
                b4.output = b4.output .- ((b4_grad/settings.batchsize)*settings.eta)

            
                w1_grad .= 0
                w2_grad .= 0
                w3_grad .= 0
                w4_grad = 0
            
                b1_grad = 0
                b2_grad = 0
                b3_grad = 0
                b4_grad = 0
            end
	end
    avg_loss = mean(train_losses)
    train_acc = train_acc/train_samples * 100
    
    @info "Train results" epoch avg_loss train_acc
end

Training... 100%|████████████████████████████████████████| Time: 0:02:50


170.659853 seconds (557.71 M allocations: 841.557 GiB, 28.20% gc time, 0.00% compilation time)


┌ Info: Train results
│   epoch = 1
│   avg_loss = 0.68412226f0
└   train_acc = 79.16
Training... 100%|████████████████████████████████████████| Time: 0:02:51


171.623584 seconds (557.69 M allocations: 841.556 GiB, 28.37% gc time)


┌ Info: Train results
│   epoch = 2
│   avg_loss = 0.29932332f0
└   train_acc = 91.14
Training... 100%|████████████████████████████████████████| Time: 0:02:54


174.555362 seconds (557.69 M allocations: 841.556 GiB, 28.45% gc time)


┌ Info: Train results
│   epoch = 3
│   avg_loss = 0.25181714f0
└   train_acc = 92.55499999999999


In [9]:
test_acc = 0;
test_losses = zeros(test_samples);

@time @showprogress dt=1 barglyphs=BarGlyphs('|','█', ['▁' ,'▂' ,'▃' ,'▄' ,'▅' ,'▆', '▇'],' ','|',) desc="Testing..." for i in 1:test_samples
    img.output = reshape(test_dataset[i].features, 28,28,1,1)
    loss_value = forward!(graph)
    test_losses[i] = loss_value
    onecold(y_output.output, 0:9) == test_dataset[i].targets ? test_acc+=1 : nothing
end
test_acc = test_acc/test_samples * 100
avg_loss = mean(test_losses)

@info "Test results" avg_loss test_acc

Testing... 100%|█████████████████████████████████████████| Time: 0:00:06


  6.928088 seconds (160.34 M allocations: 16.219 GiB, 9.73% gc time, 0.03% compilation time)


┌ Info: Test results
│   avg_loss = 7.157168446327554
└   test_acc = 93.47999999999999
